In [1]:
import datetime
import unittest

import numpy as np
import pandas as pd

In [2]:
def get_popular_products_report(orders_df, order_lines_df, n=5, dt_start=None, dt_stop=None, dt_offset=None):
    '''Return report with key product metrics
    :param orders_df: The DF with orders information
    :type orders_df: DataFrame
    :param order_lines_df: The DF with products information
    :type order_lines_df: DataFrame
    :param n: The amount of the most popular products for the report
    :type n: int
    :param dt_start: The start of the reporting interval
    :type dt_start: datetime
    :param dt_stop: The end of the reporting interval
    :type dt_stop: datetime
    :param dt_offset: The offset of the reporting interval
    :type dt_offset: DateOffset
    :returns: The report with key product metrics
    :rtype: DataFrame
    '''
    dt_stop = dt_stop or datetime.datetime.now()
    dt_offset = dt_offset or pd.DateOffset(months=1)
    dt_start = dt_start or dt_stop - dt_offset
    
    orders_info = (orders_df[(orders_df.DateTime > dt_start) & (orders_df.DateTime <= dt_stop)]
                   .merge(order_lines_df, on='OrderId'))
    orders_info = orders_info.merge(
        orders_info.groupby('OrderId', as_index=False).Price.sum().rename(columns={'Price': 'OrderPrice'}),
        on='OrderId',
    )
    
    products_info = orders_info.groupby('ProductId').agg({'Price': ['count', 'sum'], 'OrderPrice': 'mean'})
    products_info.columns = products_info.columns.droplevel()
    products_info = (products_info.sort_values(['count', 'sum'], ascending=False)
                     .rename(columns={'count': 'Quantity', 'sum': 'Revenue', 'mean': 'AvgOrderPrice'})
                     .reset_index())
    
    return products_info.nlargest(min(n, len(products_info)), 'Quantity')

In [3]:
class TestProductsReport(unittest.TestCase):
    
    def test_add(self):
        orders_data = {
            'DateTime': [
                datetime.datetime(2019, 6, 8, 12, 30),
                datetime.datetime(2019, 6, 7, 12, 30), 
                datetime.datetime(2019, 6, 7, 12, 40),
                datetime.datetime(2019, 5, 9, 12, 40),
                datetime.datetime(2019, 5, 9, 12, 30),
                datetime.datetime(2019, 5, 8, 10, 30),
                datetime.datetime(2019, 4, 8, 10, 30)
            ], 
            'CustomerId': [4, 0, 0, 2, 3, 1, 4], 
            'OrderId': [6, 0, 1, 3, 4, 2, 5],
        }
        orders_df = pd.DataFrame.from_dict(orders_data)
        
        order_lines_data = {
            'OrderId': [0, 0, 1, 2, 2, 2, 3, 4, 5, 6, 6], 
            'ProductId': [2, 3, 0, 4, 4, 2, 5, 5, 6, 7, 7], 
            'Price': [10., 100., 50., 1000., 1000., 10., 555., 555., 222., 115., 115.],
        }
        order_lines_df = pd.DataFrame.from_dict(order_lines_data)
        
        result_data = [
            (4, 2, 2000., 2010.), 
            (5, 2, 1110., 555.), 
            (2, 2, 20., 1060.), 
            (3, 1, 100., 110.),
            (0, 1, 50., 50.),
        ]
        
        result_dtypes = [
            ('ProductId','int64'),
            ('Quantity','int64'), 
            ('Revenue', 'float64'),
            ('AvgOrderPrice', 'float64'),
        ]
        
        result_df = pd.DataFrame(
            np.array(result_data, dtype=result_dtypes), 
            columns=['ProductId', 'Quantity', 'Revenue', 'AvgOrderPrice']
        )
        
        dt_start = datetime.datetime(2019, 5, 7)
        dt_stop = datetime.datetime(2019, 6, 8)
        
        pd.util.testing.assert_frame_equal(
            get_popular_products_report(orders_df, order_lines_df, dt_start=dt_start, dt_stop=dt_stop),
            result_df
        )

In [4]:
unittest.main(argv=[''], exit=False);

.
----------------------------------------------------------------------
Ran 1 test in 0.026s

OK
